In [1]:
import numpy as np

from matplotlib import pyplot as plt

from scipy.special import binom
from scipy.special import factorial as fact

%matplotlib inline

In [2]:
IGRF_COEFS_TABLE = np.genfromtxt('igrf13coeffs.txt', dtype=str)[1:]

X_CIP_COEFS_TABLES = [np.genfromtxt('tab5.2a.txt', dtype=float, skip_header=36, skip_footer=298),
                      np.genfromtxt('tab5.2a.txt', dtype=float, skip_header=1345, skip_footer=44),
                      np.genfromtxt('tab5.2a.txt', dtype=float, skip_header=1601, skip_footer=7),
                      np.genfromtxt('tab5.2a.txt', dtype=float, skip_header=1640, skip_footer=2),
                      np.genfromtxt('tab5.2a.txt', dtype=float, skip_header=1647)]

Y_CIP_COEFS_TABLES = [np.genfromtxt('tab5.2b.txt', dtype=float, skip_header=35, skip_footer=317),
                      np.genfromtxt('tab5.2b.txt', dtype=float, skip_header=1000, skip_footer=39),
                      np.genfromtxt('tab5.2b.txt', dtype=float, skip_header=1280, skip_footer=8),
                      np.genfromtxt('tab5.2b.txt', dtype=float, skip_header=1313, skip_footer=2),
                      np.genfromtxt('tab5.2b.txt', dtype=float, skip_header=1321)]

sXY_CIO_COEFS_TABLES = [np.genfromtxt('tab5.2c.txt', dtype=float, skip_header=40, skip_footer=37),
                        np.genfromtxt('tab5.2c.txt', dtype=float, skip_header=76, skip_footer=33),
                        np.genfromtxt('tab5.2c.txt', dtype=float, skip_header=82, skip_footer=7),
                        np.genfromtxt('tab5.2c.txt', dtype=float, skip_header=110, skip_footer=2),
                        np.genfromtxt('tab5.2c.txt', dtype=float, skip_header=117)]

TIP_AND_DUT1_TABLE = np.genfromtxt('6_BULLETIN_A_V2013_016.txt', dtype=float, skip_header=122, skip_footer=42)[:,3:]

# Scalar potential $V$

$$V = a \sum\limits_{n=1}^N \sum\limits_{m=0}^n \left( \dfrac{a}{r} \right)^{n+1} \left[ g^m_n(t) \cos \left( m \varphi \right) + h^m_n(t) \sin \left( m \varphi \right) \right] S P^m_n \left( \cos \theta \right)$$

where
$$a = 6371.3 \text{ km} - \text{mean equatorial radius of the Earth}$$
$$N = 13 - \text{maximum degree of expansion}$$
$$n - \text{degree of expansion}$$
$$m - \text{order of expansion}$$
$$g^m_n, h^m_n - \text{Gauss coefficients}$$
$$S P^m_n - \text{Schmidt normalized associated Legendre polynomials}$$
$$r - \text{geocentric radial distance}$$
$$\theta - \text{geocentric polar angle}$$
$$\varphi - \text{geocentric longitude}$$
$$t - \text{time}$$

# Associated Legendre polynomials

$$P^m_n \left( \cos \theta \right) = (-1)^m 2^n \left( \sin \theta \right)^m \sum\limits_{k=m}^n \dfrac{k!}{(k-m)!} \left( \cos \theta \right)^{k-m} \begin{pmatrix} n \\ k \end{pmatrix} \begin{pmatrix} \dfrac{n+k-1}{2} \\ n \end{pmatrix}$$

In [3]:
def assoc_legendre(theta, n, m):
    
    sum = 0
    
    for k in range(m, n+1):
        
        sum += fact(k) / fact(k-m) * np.cos(theta)**(k-m) * binom(n, k) * binom((n+k-1)/2, n)
        
    return (-1)**m * 2**n * np.sin(theta)**m * sum

## Differentiation by $\theta$

$$\begin{aligned}
\dfrac{\partial P^m_n \left( \cos \theta \right)}{\partial \theta} &= (-1)^m 2^n m \left( \sin \theta \right)^{m-1} \sum\limits_{k=m}^n \dfrac{k!}{(k-m)!} \left( \cos \theta \right)^{k-m+1} \begin{pmatrix} n \\ k \end{pmatrix} \begin{pmatrix} \dfrac{n+k-1}{2} \\ n \end{pmatrix} - \\
&- (-1)^m 2^n \left( \sin \theta \right)^{m+1} \sum\limits_{k=m+1}^n \dfrac{k!}{(k-m-1)!} \left( \cos \theta \right)^{k-m-1} \begin{pmatrix} n \\ k \end{pmatrix} \begin{pmatrix} \dfrac{n+k-1}{2} \\ n \end{pmatrix}
\end{aligned}$$

In [4]:
def theta_deriv(theta, n, m):
    
    SUM = 0
    sum = 0
    
    if m != 0:
    
        for k in range(m, n+1):

            sum += fact(k) / fact(k-m) * np.cos(theta)**(k-m) * binom(n, k) * binom((n+k-1)/2, n)

        SUM += m * np.sin(theta)**(m-1) * np.cos(theta) * sum

        sum = 0
    
    for k in range(m+1, n+1):
        
        sum += fact(k) / fact(k-m-1) * np.cos(theta)**(k-m-1) * (-np.sin(theta)) * binom(n, k) * binom((n+k-1)/2, n)
        
    SUM += np.sin(theta)**m * sum
    
    return (-1)**m * 2**n * SUM

## Schmidt normalization

$$\text{if } m \not = 0:$$
$$S P^m_n \left( \cos \theta \right) = (-1)^m \sqrt{\dfrac{2 (n-m)!}{(n+m)!}} P^m_n \left( \cos \theta \right)$$

In [5]:
def assoc_legendre_norm(theta, n, m):
    
    if m == 0:
        
        return assoc_legendre(theta, n, m)
    
    else:
        
        return (-1)**m * np.sqrt(2 * fact(n-m) / fact(n+m)) * assoc_legendre(theta, n, m)
    
def theta_deriv_norm(theta, n, m):
    
    if m == 0:
        
        return theta_deriv(theta, n, m)
    
    else:
        
        return (-1)**m * np.sqrt(2 * fact(n-m) / fact(n+m)) * theta_deriv(theta, n, m)

# Scalar field $V$ derivative by $\theta$

$$\dfrac{\partial V}{\partial \theta} = a \sum\limits_{n=1}^N \sum\limits_{m=0}^n \left( \dfrac{a}{r} \right)^{n+1} \left[ g^m_n(t) \cos \left( m \varphi \right) + h^m_n(t) \sin \left( m \varphi \right) \right] \dfrac{\partial S P^m_n \left( \cos \theta \right)}{\partial \theta}$$

In [6]:
def theta_deriv_full(a, G, H, r, theta, phi):
    
    N = 13
    
    sum = 0
    
    for n in range(1, N+1):
        
        for m in range(n+1):
            
            sum += (a / r)**(n+1) * (G[n,m] * np.cos(m * phi) + H[n,m] * np.sin(m * phi)) * theta_deriv_norm(theta, n, m)
            
    return sum * a

# Scalar field $V$ derivative by $r$

$$\dfrac{\partial V}{\partial r} = - \sum\limits_{n=1}^N \sum\limits_{m=0}^n (n+1) \left( \dfrac{a}{r} \right)^{n+2} \left[ g^m_n(t) \cos \left( m \varphi \right) + h^m_n(t) \sin \left( m \varphi \right) \right] S P^m_n \left( \cos \theta \right)$$

In [7]:
def r_deriv_full(a, G, H, r, theta, phi):
    
    N = 13
    
    sum = 0
    
    for n in range(1, N+1):
        
        for m in range(n+1):
            
            sum += (n+1) * (a / r)**(n+2) * (G[n,m] * np.cos(m * phi) + H[n,m] * np.sin(m * phi)) * \
                    assoc_legendre_norm(theta, n, m)
            
    return sum * (-1)

# Scalar field $V$ derivative by $\varphi$

$$\dfrac{\partial V}{\partial \varphi} = a \sum\limits_{n=1}^N \sum\limits_{m=0}^n \left( \dfrac{a}{r} \right)^{n+1} m \left[ h^m_n(t) \cos \left( m \varphi \right) - g^m_n(t) \sin \left( m \varphi \right) \right] S P^m_n \left( \cos \theta \right)$$

In [8]:
def phi_deriv_full(a, G, H, r, theta, phi):
    
    N = 13
    
    sum = 0
    
    for n in range(1, N+1):
        
        for m in range(n+1):
            
            sum += (a / r)**(n+1) * m * (H[n,m] * np.cos(m * phi) - G[n,m] * np.sin(m * phi)) * assoc_legendre_norm(theta, n, m)
            
    return sum * a

# Coefficients $g^m_n (t)$ and $h^m_n (t)$

In [9]:
def IGRF_coef(year, table=IGRF_COEFS_TABLE):
    
    if (year < 1900.0) or (year >= 2025.0):
        raise Exception('Invalid year, should be in [1900, 2025)')
        
    delta_year = year % 5
    year_5 = year - delta_year
    
    G = np.zeros((14, 14))
    H = np.zeros((14, 14))
    
    if year_5 < 2020:
    
        col_num = int(year_5 / 5 - 377)
        
        coefs = table[:,(0,1,2,col_num,col_num+1)]
        
        for line in coefs[1:]:
            
            val = float(line[3]) + delta_year / 5 * (float(line[4]) - float(line[3]))
            
            if line[0] == 'g':
                
                G[int(line[1]), int(line[2])] = val
            
            elif line[0] == 'h':
                
                H[int(line[1]), int(line[2])] = val
                
            else:
                
                raise Exception('Incorrect symbol in line, should be \'g/h\'')
                
    else:
        
        coefs = table[:,(0,1,2,-2,-1)]
        
        for line in coefs[1:]:
            
            val = float(line[3]) + delta_year * float(line[4])
            
            if line[0] == 'g':
                
                G[int(line[1]), int(line[2])] = val
            
            elif line[0] == 'h':
                
                H[int(line[1]), int(line[2])] = val
                
            else:
                
                raise Exception('Incorrect symbol in line, should be \'g/h\'')
        
    return G, H

# Magnetic field

$$\mathbf{B} = - \nabla V = \begin{pmatrix} -\dfrac{1}{r} \dfrac{\partial V}{\partial \theta} & -\dfrac{1}{r \sin \theta} \dfrac{\partial V}{\partial \varphi} & -\dfrac{\partial V}{\partial r} \end{pmatrix}^{\top}$$

Reference frame: SouthEastUp

In [10]:
def magn_field(year, r, theta, phi, table=IGRF_COEFS_TABLE):
    
    a = 6371200 # [m]
    
    G, H = IGRF_coef(year, table=table)
    
    B_r = - r_deriv_full(a, G, H, r, theta, phi)
    B_theta = - theta_deriv_full(a, G, H, r, theta, phi) / r
    B_phi = - phi_deriv_full(a, G, H, r, theta, phi) / (r * np.sin(theta))
    
    return np.array([B_theta, B_phi, B_r])

In [11]:
B = magn_field(2020, 6371200, np.pi/3, np.pi/2)

print('Magnetic fields components in South-East-Up geographical frame:', B, 'nT')
print('Intensity of the field:', np.linalg.norm(B), 'nT')

Magnetic fields components in South-East-Up geographical frame: [-33961.96953674     81.30084729 -37372.52645732] nT
Intensity of the field: 50498.7892770229 nT


# Conversion to Earth-Centered Earth-Fixed frame

$$\mathbf{B}_\text{ECEF} = \begin{pmatrix} \cos{\theta} \cos{\varphi} & - \sin{\varphi} & \sin{\theta} \cos{\varphi} \\ \cos{\theta} \sin{\varphi} & \cos{\varphi} & \sin{\theta} \sin{\varphi} \\ - \sin{\theta} & 0 & \cos{\theta} \end{pmatrix} \mathbf{B}_\text{SEU}$$

In [12]:
def DCM_SEU_to_ECEF(theta, phi):
    
    return np.array([[np.cos(theta) * np.cos(phi), -np.sin(phi), np.sin(theta) * np.cos(phi)],
                     [np.cos(theta) * np.sin(phi), np.cos(phi), np.sin(theta) * np.sin(phi)],
                     [-np.sin(theta), 0, np.cos(theta)]])

def magn_field_ECEF(year, r, theta, phi, table=IGRF_COEFS_TABLE):
    
    B = magn_field(year, r, theta, phi, table=table)
    
    A = DCM_SEU_to_ECEF(theta, phi)
    
    return A @ B

In [13]:
B_ECEF = magn_field_ECEF(2020, 6371200, np.pi/3, np.pi/2)

print('Magnetic fields components in ECEF:', B_ECEF, 'nT')

Magnetic fields components in ECEF: [   -81.30084729 -49346.54208401  10725.66515271] nT


# Conversion to Earth-Centered Inertial frame

## Motion of the celestial pole in celestial reference frame

### Fundamental arguments of the nutation theory

pp 67-68 of the IERS Conventions

* Mean Anomaly of the Moon $l$

$$F_1 = 134.96340251^{\circ} + 1717915923.2178'' \cdot T + 31.8792'' \cdot T^2 + 0.051635'' \cdot T^3 - 0.00024470'' \cdot T^4$$


* Mean Anomaly of the Sun $l'$

$$F_2 = 357.52910918^{\circ} + 129596581.0481'' \cdot T - 0.5532'' \cdot T^2 + 0.000136'' \cdot T^3 - 0.00001149'' \cdot T^4$$


* Mean Longitude of the Moon $-$ Mean Longitude of the Ascending Node of the Moon $F = L - \Omega$

$$F_3 = 93.27209062^{\circ} + 1739527262.8478'' \cdot T - 12.7512'' \cdot T^2 - 0.001037'' \cdot T^3 + 0.00000417'' \cdot T^4$$


* Mean Elongation of the Moon from the Sun $D$

$$F_4 = 297.85019547^{\circ} + 1602961601.2090'' \cdot T - 6.3706'' \cdot T^2 + 0.006593'' \cdot T^3 - 0.00003169'' \cdot T^4$$


* Mean Longitude of the Ascending Node of the Moon $\Omega$

$$F_5 = 125.04455501^{\circ} - 6962890.5431'' \cdot T + 7.4722'' \cdot T^2 + 0.007702'' \cdot T^3 - 0.00005939'' \cdot T^4$$


* Mean Longitude of the Mercury $L_\text{Me}$

$$F_6 = 4.402608842 + 2608.7903141574 \cdot T$$


* Mean Longitude of the Venus $L_\text{Ve}$

$$F_7 = 3.176146697 + 1021.3285546211 \cdot T$$


* Mean Longitude of the Earth $L_\text{E}$

$$F_8 = 1.753470314 + 628.3075849991 \cdot T$$


* Mean Longitude of the Mars $L_\text{Ma}$

$$F_9 = 6.203480913 + 334.0612426700 \cdot T$$


* Mean Longitude of the Jupiter $L_\text{Ju}$

$$F_{10} = 0.599546497 + 52.9690962641 \cdot T$$


* Mean Longitude of the Saturn $L_\text{Sa}$

$$F_{11} = 0.874016757 + 21.3299104960 \cdot T$$


* Mean Longitude of the Uranus $L_\text{Ur}$

$$F_{12} = 5.481293872 + 7.4781598567 \cdot T$$


* Mean Longitude of the Neptune $L_\text{Ne}$

$$F_{13} = 5.311886287 + 3.8133035638 \cdot T$$


* General precession $p_A$

$$F_{14} = 0.02438175 \cdot T + 0.00000538691 \cdot T^2$$

where $T$ is the Julian Century

In [55]:
def F1(JC):
    
    """
    l (Mean Anomaly of the Moon) in [rad]
    """
    
    DEG2RAD = np.pi / 180
    ARCSEC2RAD = DEG2RAD / 3600
    
    return 134.96340251 * DEG2RAD + \
           (1717915923.2178 * JC + 31.8792 * JC**2 + 0.051635 * JC**3 - 0.00024470 * JC**4) * ARCSEC2RAD

def F2(JC):
    
    """
    l' (Mean Anomaly of the Sun) in [rad]
    """
    
    DEG2RAD = np.pi / 180
    ARCSEC2RAD = DEG2RAD / 3600
    
    return 357.52910918 * DEG2RAD + \
           (129596581.0481 * JC - 0.5532 * JC**2 + 0.000136 * JC**3 - 0.00001149 * JC**4) * ARCSEC2RAD

def F3(JC):
    
    """
    F = L - Omega (Mean Longitude of the Moon - Mean Longitude of the Ascending Node of the Moon) in [rad]
    """
    
    DEG2RAD = np.pi / 180
    ARCSEC2RAD = DEG2RAD / 3600
    
    return 93.27209062 * DEG2RAD + \
           (1739527262.8478 * JC - 12.7512 * JC**2 - 0.001037 * JC**3 + 0.00000417 * JC**4) * ARCSEC2RAD

def F4(JC):
    
    """
    D (Mean Elongation of the Moon from the Sun) in [rad]
    """
    
    DEG2RAD = np.pi / 180
    ARCSEC2RAD = DEG2RAD / 3600
    
    return 297.85019547 * DEG2RAD + \
           (1602961601.2090 * JC - 6.3706 * JC**2 + 0.006593 * JC**3 - 0.00003169 * JC**4) * ARCSEC2RAD

def F5(JC):
    
    """
    Omega (Mean Longitude of the Ascending Node of the Moon) in [rad]
    """
    
    DEG2RAD = np.pi / 180
    ARCSEC2RAD = DEG2RAD / 3600
    
    return 125.04455501 * DEG2RAD + \
           (-6962890.5431 * JC + 7.4722 * JC**2 + 0.007702 * JC**3 - 0.00005939 * JC**4) * ARCSEC2RAD

def F6(JC):
    
    """
    L_me (Mean Longitude of the Mercury) in [rad]
    """
    
    return 4.402608842 + 2608.7903141574 * JC

def F7(JC):
    
    """
    L_ve (Mean Longitude of the Venus) in [rad]
    """
    
    return 3.176146697 + 1021.3285546211 * JC

def F8(JC):
    
    """
    L_e (Mean Longitude of the Earth) in [rad]
    """
    
    return 1.753470314 + 628.3075849991 * JC

def F9(JC):
    
    """
    L_ma (Mean Longitude of the Mars) in [rad]
    """
    
    return 6.203480913 + 334.0612426700 * JC

def F10(JC):
    
    """
    L_ju (Mean Longitude of the Jupiter) in [rad]
    """
    
    return 0.599546497 + 52.9690962641 * JC

def F11(JC):
    
    """
    L_sa (Mean Longitude of the Saturn) in [rad]
    """
    
    return 0.874016757 + 21.3299104960 * JC

def F12(JC):
    
    """
    L_ur (Mean Longitude of the Uranus) in [rad]
    """
    
    return 5.481293872 + 7.4781598567 * JC

def F13(JC):
    
    """
    L_ne (Mean Longitude of the Neptune) in [rad]
    """
    
    return 5.311886287 + 3.8133035638 * JC

def F14(JC):
    
    """
    pA (General precession) in [rad]
    """
    
    return 0.02438175 * JC + 0.00000538691 * JC**2

### Nutation argument $\beta$

$$\beta = \sum\limits_{i=1}^{14} N_i F_i$$

where

$$F_i - \text{fundamental arguments of the nutation theory}$$

$$N_i - \text{Poisson multipliers (tables 5.2a, 5.2b, 5.2c at  ftp://tai.bipm.org/iers/conv2003/chapter5/)}$$

In [56]:
def nutation_argument(N, JC):
    
    F = np.array([F1(JC), F2(JC), F3(JC), F4(JC), F5(JC),
                  F6(JC), F7(JC), F8(JC), F9(JC), F10(JC), F11(JC), F12(JC), F13(JC), F14(JC)])
    
    return N @ F

### Celestial Intermediate Point coordinates in the GCRS (X)

p. 54 of the IERS Conventions

$$X = X_\text{polynomial} + X_\text{non-polynomial}$$

$$X_\text{polynomial} [\mu\text{as}] = -16616.99 + 2004191742.88 \cdot T - 427219.05 \cdot T^2 - 198620.54 \cdot T^3 - 46.05 \cdot T^4 + 5.98  \cdot T^5$$

$$X_\text{non-polynomial} [\mu\text{as}] = \sum\limits_{j=0}^4 \sum\limits_{i} \left[ \left( a_{s,j} \right)_i \sin{\beta} + \left( a_{c,j} \right)_i \cos{\beta} \right] T^j$$

where

$$T - \text{Julian Centuries}$$

$$\beta - \text{nutation argument (Poisson terms } \left(N_j\right)_i \text{ are taken from table 5.2a)}$$

$$\left( a_{s,j} \right)_i \text{ and } \left( a_{c,j} \right)_i - \text{Fourier terms (taken from table 5.2a)}$$

In [57]:
def X_polynom(JC):
    
    """
    Polynomial part of the X-coordinate of the CIP in [uas]
    """
    
    return -16616.99 + 2004191742.88 * JC - 427219.05 * JC**2 - 198620.54 * JC**3 - 46.05 * JC**4 + 5.98 * JC**5

def X_non_polynom(JC, tables=X_CIP_COEFS_TABLES):
    
    """
    Non-polynomial part of the X-coordinate of the CIP in [uas]
    """
    
    coefs_0, coefs_1, coefs_2, coefs_3, coefs_4 = tables

    a_s0 = coefs_0[:,1]
    a_s1 = coefs_1[:,1]
    a_s2 = coefs_2[:,1]
    a_s3 = coefs_3[:,1]
    a_s4 = coefs_4[1].reshape(1,)
    a_sj = [a_s0, a_s1, a_s2, a_s3, a_s4]

    a_c0 = coefs_0[:,2]
    a_c1 = coefs_1[:,2]
    a_c2 = coefs_2[:,2]
    a_c3 = coefs_3[:,2]
    a_c4 = coefs_4[2].reshape(1,)
    a_cj = [a_c0, a_c1, a_c2, a_c3, a_c4]

    N0 = coefs_0[:,3:]
    N1 = coefs_1[:,3:]
    N2 = coefs_2[:,3:]
    N3 = coefs_3[:,3:]
    N4 = coefs_4[3:].reshape(1,14)
    Nj = [N0, N1, N2, N3, N4]
    
    SUM = 0
    
    for (j, (a_s, a_c, N)) in enumerate(zip(a_sj, a_cj, Nj)):
        
        beta = nutation_argument(N, JC)
        
        for (a_si, a_ci, betai) in zip(a_s, a_c, beta):
            
            SUM += (a_si * np.sin(betai) + a_ci * np.cos(betai)) * JC**j
            
    return SUM

def X_CIP(JC, tables=X_CIP_COEFS_TABLES):
    
    """
    X-coordinate of the CIP in [rad]
    """
    
    UARCSEC2RAD = np.pi / 180 / 3600 / 1000000
    
    return (X_polynom(JC) + X_non_polynom(JC, tables=tables)) * UARCSEC2RAD

### Celestial Intermediate Point coordinates in the GCRS (Y)

p. 54 of the IERS Conventions

$$Y = Y_\text{polynomial} + Y_\text{non-polynomial}$$

$$Y_\text{polynomial} [\mu\text{as}] = -6950.78 - 25381.99 T - 22407250.99 T^2 + 1842.28 T^3 + 1113.06 T^4 + 0.99 T^5$$

$$Y_\text{non-polynomial} [\mu\text{as}] = \sum\limits_{j=0}^4 \sum\limits_{i} \left[ \left( b_{s,j} \right)_i \sin{\beta} + \left( b_{c,j} \right)_i \cos{\beta} \right] T^j$$

where

$$T - \text{Julian Centuries}$$

$$\beta - \text{nutation argument (Poisson terms } \left(N_j\right)_i \text{ are taken from table 5.2b)}$$

$$\left( b_{s,j} \right)_i \text{ and } \left( b_{c,j} \right)_i - \text{Fourier terms (taken from table 5.2b)}$$

In [58]:
def Y_polynom(JC):
    
    """
    Polynomial part of the Y-coordinate of the CIP in [uas]
    """
    
    return -6950.78 - 25381.99 * JC - 22407250.99 * JC**2 + 1842.28 * JC**3 + 1113.06 * JC**4 + 0.99 * JC**5

def Y_non_polynom(JC, tables=Y_CIP_COEFS_TABLES):
    
    """
    Non-polynomial part of the Y-coordinate of the CIP in [uas]
    """
    
    coefs_0, coefs_1, coefs_2, coefs_3, coefs_4 = tables

    b_s0 = coefs_0[:,1]
    b_s1 = coefs_1[:,1]
    b_s2 = coefs_2[:,1]
    b_s3 = coefs_3[:,1]
    b_s4 = coefs_4[1].reshape(1,)
    b_sj = [b_s0, b_s1, b_s2, b_s3, b_s4]

    b_c0 = coefs_0[:,2]
    b_c1 = coefs_1[:,2]
    b_c2 = coefs_2[:,2]
    b_c3 = coefs_3[:,2]
    b_c4 = coefs_4[2].reshape(1,)
    b_cj = [b_c0, b_c1, b_c2, b_c3, b_c4]

    N0 = coefs_0[:,3:]
    N1 = coefs_1[:,3:]
    N2 = coefs_2[:,3:]
    N3 = coefs_3[:,3:]
    N4 = coefs_4[3:].reshape(1,14)
    Nj = [N0, N1, N2, N3, N4]
    
    SUM = 0
    
    for (j, (b_s, b_c, N)) in enumerate(zip(b_sj, b_cj, Nj)):
        
        beta = nutation_argument(N, JC)
        
        for (b_si, b_ci, betai) in zip(b_s, b_c, beta):
            
            SUM += (b_si * np.sin(betai) + b_ci * np.cos(betai)) * JC**j
            
    return SUM

def Y_CIP(JC, tables=Y_CIP_COEFS_TABLES):
    
    """
    Y-coordinate of the CIP in [rad]
    """
    
    UARCSEC2RAD = np.pi / 180 / 3600 / 1000000
    
    return (Y_polynom(JC) + Y_non_polynom(JC, tables=tables)) * UARCSEC2RAD

### Celestial Intermediate Origin locator $s$

p. 59 of the IERS Conventions

$$s + \dfrac{XY}{2} = \left(s + \dfrac{XY}{2}\right)_\text{polynomial} + \left(s + \dfrac{XY}{2}\right)_\text{non-polynomial}$$

$$\left(s + \dfrac{XY}{2}\right)_\text{polynomial} [\mu\text{as}] = 94.0 + 3808.35 T - 119.94 T^2 - 72574.09 T^3 + 27.70 T^4 + 15.61 T^5$$

$$\left(s + \dfrac{XY}{2}\right)_\text{non-polynomial} [\mu\text{as}] = \sum\limits_{j=0}^4 \sum\limits_{i} \left[ \left( c_{s,j} \right)_i \sin{\beta} + \left( c_{c,j} \right)_i \cos{\beta} \right] T^j$$

where

$$T - \text{Julian Centuries}$$

$$X \text{ and } Y - \text{CIP coordinates}$$

$$\beta - \text{nutation argument (Poisson terms } \left(N_j\right)_i \text{ are taken from table 5.2c)}$$

$$\left( c_{s,j} \right)_i \text{ and } \left( c_{c,j} \right)_i - \text{Fourier terms (taken from table 5.2c)}$$

In [59]:
def sXY_polynom(JC):
    
    """
    Polynomial part of the s + XY/2 quantity in [uas]
    """
    
    return -6950.78 - 25381.99 * JC - 22407250.99 * JC**2 + 1842.28 * JC**3 + 1113.06 * JC**4 + 0.99 * JC**5

def sXY_non_polynom(JC, tables=sXY_CIO_COEFS_TABLES):
    
    """
    Non-polynomial part of the s + XY/2 quantity in [uas]
    """
    
    coefs_0, coefs_1, coefs_2, coefs_3, coefs_4 = tables

    c_s0 = coefs_0[:,1]
    c_s1 = coefs_1[:,1]
    c_s2 = coefs_2[:,1]
    c_s3 = coefs_3[:,1]
    c_s4 = coefs_4[1].reshape(1,)
    c_sj = [c_s0, c_s1, c_s2, c_s3, c_s4]

    c_c0 = coefs_0[:,2]
    c_c1 = coefs_1[:,2]
    c_c2 = coefs_2[:,2]
    c_c3 = coefs_3[:,2]
    c_c4 = coefs_4[2].reshape(1,)
    c_cj = [c_c0, c_c1, c_c2, c_c3, c_c4]

    N0 = coefs_0[:,3:]
    N1 = coefs_1[:,3:]
    N2 = coefs_2[:,3:]
    N3 = coefs_3[:,3:]
    N4 = coefs_4[3:].reshape(1,14)
    Nj = [N0, N1, N2, N3, N4]
    
    SUM = 0
    
    for (j, (c_s, c_c, N)) in enumerate(zip(c_sj, c_cj, Nj)):
        
        beta = nutation_argument(N, JC)
        
        for (c_si, c_ci, betai) in zip(c_s, c_c, beta):
            
            SUM += (c_si * np.sin(betai) + c_ci * np.cos(betai)) * JC**j
            
    return SUM

def sXY(JC, tables=sXY_CIO_COEFS_TABLES):
    
    """
    s + XY/2 quantity in [rad]
    """
    
    UARCSEC2RAD = np.pi / 180 / 3600 / 1000000
    
    return (sXY_polynom(JC) + sXY_non_polynom(JC, tables=tables)) * UARCSEC2RAD

## Nutation-Precession matrix $\mathbf{Q}$

p. 49 of the IERS Conventions

$$\mathbf{Q} = \begin{pmatrix} 1 - a X^2 & -a XY & X \\ -a XY & 1 - a Y^2 & Y \\ -X & -Y & 1 - a \left( X^2 + Y^2 \right) \end{pmatrix} R_3 (s)$$

where

$$X \text{ and } Y - \text{CIP coordinates}$$

$$s - \text{CIO locator}$$

$$a = \dfrac{1}{2} + \dfrac{1}{8} \left( X^2 + Y^2 \right) \text{ with accuracy of 1} \mu \text{as}$$

$$R_3 (s) = \begin{pmatrix} \cos{s} & \sin{s} & 0 \\ -\sin{s} & \cos{s} & 0 \\ 0 & 0 & 1 \end{pmatrix}$$

In [60]:
def nut_prec(JC, tables_X=X_CIP_COEFS_TABLES, tables_Y=Y_CIP_COEFS_TABLES, tables_sXY=sXY_CIO_COEFS_TABLES):
    
    X = X_CIP(JC, tables=tables_X)
    Y = Y_CIP(JC, tables=tables_Y)
    s = sXY(JC, tables=tables_sXY) - X*Y/2
    
    X2Y2 = X**2 + Y**2
    XY = X*Y
    
    a = 0.5 + X2Y2 / 8
    
    A = np.array([[1 - a*X**2, -a*XY, X],
                  [-a*XY, 1 - a*Y**2, Y],
                  [-X, -Y, 1 - a*X2Y2]])
    
    R = np.array([[np.cos(s), np.sin(s), 0],
                  [-np.sin(s), np.cos(s), 0],
                  [0, 0, 1]])
    
    return A @ R

## Time Conversions

### Gregorian date to Julian date

$$\text{JD} = 1720981.5 + \left[ 365.25 Y \right] + \left[ 30.6001 \left( M + 1 \right) \right] + \text{Day} + \dfrac{\text{Hour}}{24} + \dfrac{\text{Min}}{24 \cdot 60} + \dfrac{\text{Sec}}{24 \cdot 3600}$$

where

$$\left\{
\begin{aligned}
&\left[ \begin{aligned}
Y &= \text{Year} - 1,\\
M &= \text{Month} + 12,
\end{aligned} \right. \qquad \text{if Month} \leqslant 2,
\\
&\left[ \begin{aligned}
Y &= \text{Year},\\
M &= \text{Month}.
\end{aligned} \right. \qquad \qquad \text{if Month} > 2.
\end{aligned}\right.
$$

In [61]:
def greg_to_julian(GD):
    
    year, month, day, hour, min, sec = GD
    
    if month <= 2:
        
        year -= 1
        month += 12
        
    return int(365.25 * year) + int(30.6001 * (month + 1)) + day + 1720981.5 + hour / 24 + min / 24 / 60 + sec / 24 / 3600
    
print(greg_to_julian((2020, 3, 31, 16, 38, 29.36342400000285)))

2458940.1933954097


### Julian date to Gregorian date

$$b = \left[ \text{JD} + 0.5 \right] + 1537$$

$$c = \left[ \dfrac{b - 122.1}{365.25} \right]$$

$$d = \left[ 365.25 c \right]$$

$$e = \left[ \dfrac{b - d}{30.6001} \right]$$

$$f = \left( \text{JD} + 0.5 - \left[ \text{JD} + 0.5 \right] \right)$$

$$\text{Hour} = \left[ 24 f \right]$$

$$\text{Min} = \left[ 60 \cdot \left( 24 f - \text{Hour} \right) \right]$$

$$\text{Sec} = \left[ 60 \cdot \left( 60 \cdot \left( 24 f - \text{Hour} \right) - \text{Min} \right) \right]$$

$$\text{Day} = b - d - \left[ 30.6001 e \right]$$

$$\text{Month} = e - 1 - 12 \cdot \left[ \dfrac{e}{14} \right]$$

$$\text{Year} = c - 4715 - \left[ \dfrac{7 + \text{Month}}{10} \right]$$

In [62]:
def julian_to_greg(JD):
    
    b = int(JD + 0.5) + 1537
    c = int((b - 122.1) / 365.25)
    d = int(365.25 * c)
    e = int((b - d) / 30.6001)
    
    hour = (JD + 0.5 - int(JD + 0.5)) * 24
    min = (hour - int(hour)) * 60
    hour = int(hour)
    sec = (min - int(min)) * 60
    min = int(min)
    day = b - d - int(30.6001 * e)
    month = e - 1 - 12 * int(e / 14)
    year = c - 4715 - int((month + 7) / 10)
    
    return (year, month, day, hour, min, sec)

print(julian_to_greg(2.458928659722222e+06))

(2020, 3, 20, 3, 49, 59.99998211860657)


### Julian date to fraction of the Gregorian year

In [63]:
def julian_to_year_frac(JD):
    
    year, month, day, hour, min, sec = julian_to_greg(JD)
    
    if year % 4 == 0:
        DAYS = {0:0, 1:31, 2:60, 3:91, 4:121, 5:152, 6:182, 7:213, 8:244, 9:274, 10:305, 11:335, 12:366}
    else:
        DAYS = {0:0, 1:31, 2:59, 3:90, 4:120, 5:151, 6:181, 7:212, 8:243, 9:273, 10:304, 11:334, 12:365}
        
    return year + (DAYS[month-1] + day) / DAYS[12] + (hour + min / 60 + sec / 3600 ) / DAYS[12] / 24

### Modified Julian date

In [64]:
def mjd(JD):
    
    return JD - 2400000.5

### Julian centuries, starting from 2000 Jan 1d 12h, UTC

$$T = \dfrac{\text{JD} - 2451545}{36525}$$

In [65]:
def jul_cent(JD):
    
    """
    Julian Centuries, starting from 2000 Jan 1d 12h, in UTC
    """
    
    return (JD - 2451545) / 36525

In [66]:
def JD_add_seconds(seconds):
    
    return seconds / 24 / 3600

### Julian dates, starting from 2000 Jan 1d 12h, UT1

Relation between $\text{UT1}$ and $\text{UTC}$:

$$\text{UT1} = \text{UTC} + \text{dUT1}$$

where $\text{dUT1}$ is taken from IERS Bulletin A. Since $\text{dUT1}$ is usually in seconds:

$$\text{JD}_{\text{UT1}} = \text{JD}_{\text{UTC}} + \dfrac{\text{dUT1}}{24 \cdot 3600}$$

Julian dates in $\text{UT1}$ (minus 2000 Jan 1d 12h):

$$T_u = \text{JD}_{\text{UT1}} - 2451545$$

$$T_u = 36525 T + \dfrac{\text{dUT1}}{24 \cdot 3600}$$

where

$$T - \text{Julian centuries in UTC, starting from 2000 Jan 1d 12h}$$

In [67]:
def jul_date_ut1(JC, DUT1):
    
    """
    Julian date, starting from 2000 Jan 1d 12h, in UT1
    based on Julian Centuries in UTC
    """
    
    return 36525 * JC + DUT1 / 24 / 3600

### Earth-Rotation Angle $\vartheta$

$$\vartheta = 2 \pi \left( 0.7790572732640 + 1.00273781191135448 T_u \right)$$

where

$$T_u - \text{Julian date in UT1, starting from 2000 Jan 1d 12h}$$

In [68]:
def ERA(JC, DUT1):
    
    """
    Earth-Roation Angle in [rad]
    """
    
    return 2 * np.pi * (0.7790572732640 + 1.00273781191135448 * jul_date_ut1(JC, DUT1))

## Earth-Rotation matrix $\mathbf{R}$

p. 48 of the IERS Conventions

$$\mathbf{R} = \begin{pmatrix} \cos{\vartheta} & -\sin{\vartheta} & 0 \\ \sin{\vartheta} & \cos{\vartheta} & 0 \\ 0 & 0 & 1 \end{pmatrix}$$

In [69]:
def earth_rot(JC, DUT1):
    
    vartheta = ERA(JC, DUT1)
    
    return np.array([[np.cos(vartheta), -np.sin(vartheta), 0],
                     [np.sin(vartheta), np.cos(vartheta), 0],
                     [0, 0, 1]])

## Polar motion

### Terrestrial Intermediate Origin locator $s'$

p. 52 of the IERS Conventions

$$s' = -47 \mu \text{as} T$$

where

$$T - \text{Julian Centuries}$$

In [70]:
def tio_loc(JC):
    
    """
    TIO locator s' in [rad]
    """
    
    UARCSEC2RAD = np.pi / 180 / 3600 / 1000000
    
    return -47 * UARCSEC2RAD * JC

### Terrestrial Intermediate Point coordinates in the ITRS $\left(x_p, y_p \right)$

IERS Bulletin A

Currently, the file is suitable for dates 27.03.2020 - 26.03.2021

Coordinates are updated only for days

In [71]:
def TIP_coords(JD, table=TIP_AND_DUT1_TABLE):
    
    """
    returns:
            x_p in [rad]
            y_p in [rad]
            dUT1 in [sec]
    """
    
    ARCSEC2RAD = np.pi / 180 / 3600
    
    MJD = int(mjd(JD))
    
    if MJD > 59299 or MJD < 58935:
        
        raise Exception('Non-sustainable date, import another file')
    
    num = np.argwhere(table[:,0] == MJD)[0,0]
    
    x_p, y_p, DUT1 = table[num, 1:]
    
    return np.array([x_p * ARCSEC2RAD, y_p * ARCSEC2RAD, DUT1])

## Polar motion matrix $\mathbf{W}$

$$\mathbf{W} = R_3 \left( -s' \right) R_2 \left( x_p \right) R_1 \left( y_p \right)$$

where

$$x_p \text{ and } y_p - \text{TIP coordinates}$$

$$s' - \text{TIO locator}$$

$$R_3 \left( -s' \right) = \begin{pmatrix} \cos{s'} & -\sin{s'} & 0 \\ \sin{s'} & \cos{s'} & 0 \\ 0 & 0 & 1 \end{pmatrix}$$

$$R_2 \left( x_p \right) = \begin{pmatrix} \cos{x_p} & 0 & -\sin{x_p} \\ 0 & 1 & 0 \\ \sin{x_p} & 0 & \cos{x_p} \end{pmatrix}$$

$$R_1 \left( y_p \right) = \begin{pmatrix} 1 & 0 & 0 \\ 0 & \cos{y_p} & \sin{y_p} \\ 0 & -\sin{y_p} & \cos{y_p} \end{pmatrix}$$

In [72]:
def polar_mot(x_p, y_p, JC):
    
    s = tio_loc(JC)
    
    R3 = np.array([[np.cos(s), -np.sin(s), 0],
                   [np.sin(s), np.cos(s), 0],
                   [0, 0, 1]])
    
    R2 = np.array([[np.cos(x_p), 0, -np.sin(x_p)],
                   [0, 1, 0],
                   [np.sin(x_p), 0, np.cos(x_p)]])
    
    R1 = np.array([[1, 0, 0],
                   [0, np.cos(y_p), np.sin(y_p)],
                   [0, -np.sin(y_p), np.cos(y_p)]])
    
    return R3 @ R2 @ R1

## ECEF to ECI rotation matrix

$$\mathbf{B}_\text{ECI} = \mathbf{Q} \mathbf{R} \mathbf{W} \mathbf{B}_\text{ECEF}$$

In [73]:
def DCM_ECEF_to_ECI(JD_UTC, tables_X=X_CIP_COEFS_TABLES, tables_Y=Y_CIP_COEFS_TABLES, tables_sXY=sXY_CIO_COEFS_TABLES,
                    table_TIP_and_DUT1=TIP_AND_DUT1_TABLE):
    
    JD_TT = JD_UTC + JD_add_seconds(37 + 32.184)
    JC_UTC = jul_cent(JD_UTC)
    JC_TT = jul_cent(JD_TT)
    x_p, y_p, DUT1 = TIP_coords(JD_UTC, table=table_TIP_and_DUT1)
    
    Q = nut_prec(JC_TT, tables_X=tables_X, tables_Y=tables_Y, tables_sXY=tables_sXY)
    R = earth_rot(JC_UTC, DUT1)
    W = polar_mot(x_p, y_p, JC_TT)
    
    return Q @ R @ W

def magn_field_ECI(JD_UTC, r, theta, phi, table_IGRF=IGRF_COEFS_TABLE, tables_X=X_CIP_COEFS_TABLES,
                   tables_Y=Y_CIP_COEFS_TABLES, tables_sXY=sXY_CIO_COEFS_TABLES, table_TIP_and_DUT1=TIP_AND_DUT1_TABLE):
    
    year = julian_to_year_frac(JD_UTC)
    B = magn_field_ECEF(year, r, theta, phi, table=table_IGRF)
    
    A = DCM_ECEF_to_ECI(JD_UTC, tables_X=tables_X, tables_Y=tables_Y, tables_sXY=tables_sXY,
                        table_TIP_and_DUT1=table_TIP_and_DUT1)
    
    return A @ B

In [33]:
B_ECI = magn_field_ECI(2458940.1933954097, 6371200, np.pi/3, np.pi/2)

print('Magnetic fields components in ECI:', B_ECI, 'nT')

Magnetic fields components in ECI: [48451.61671127 -9601.25289663 10613.82402944] nT
